<a href="https://colab.research.google.com/github/suprech/class2025Spring/blob/main/nlp_class2025spring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [84]:
from google.colab import drive
#drive.mount('/content/drive')

In [85]:
!pip install selenium
!apt-get update

# (최초 1회)
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver '/content/drive/MyDrive/Colab Notebooks' #
# !pip install chromedriver-autoinstaller

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu jammy-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:6 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:7 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Reading package lists... Done
W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)


In [86]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import re
import requests
from bs4 import BeautifulSoup

# Google Patent 기본 URL(예시 : https://patents.google.com/patent/US20250058223/)
base_url = "https://patents.google.com/patent"

# 추출 대상 특허 공개번호(출원번호)
# search_publication_number = "US20250058223"
# search_publication_number = "KR101124299B1"
search_publication_number = "US20220336530A1"

search_url = f"{base_url}/{search_publication_number}"


Beautiful Soup 설정

In [87]:
# User-Agent 설정
headers = {
    "User-Agent": "Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) "
    "AppleWebKit/537.36 (KHTML, like Gecko)"
}

# 페이지 요청
response = requests.get(search_url, headers=headers)
response.encoding = "utf-8"

# HTML 파싱
soup = BeautifulSoup(response.text, "html.parser")

Selenium

In [88]:
# selenium 관련
# headless 모드 설정
chrome_options = Options()
chrome_options.add_argument("--headless")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--disable-dev-shm-usage")

# 크롬드라이버로 원하는 url로 접속 (headless 모드)
driver = webdriver.Chrome(options=chrome_options)
driver.get(search_url)

In [89]:
# CSS 선택자 - 청구항 개수 관련
claim_selector = (
    "#claims > h3 > "
    "div:nth-child(1) > "
    "div.flex.style-scope.patent-result > "
    "span"
)

# CSS 선택자 - 도면 개수 관련
drawings_selector = "#thumbnails > h3 > span"

# 청구항 개수 및 도면 개수 추출 - 요소가 나타날 때까지 최대 3초 기다림
# wait = WebDriverWait(driver, 3)
# claim_count = wait.until(
#     EC.presence_of_element_located((By.CSS_SELECTOR, claim_selector))
# )

claim_count = driver.find_element(By.CSS_SELECTOR, claim_selector)
drawing_count = driver.find_element(By.CSS_SELECTOR, drawings_selector)

# 청구항 총 개수 추출
claim_count_text = claim_count.text
claim_count_number = re.search(r'\d+', claim_count.text).group()
print(f"청구항 개수: {claim_count_number}")

# 도면 총 개수 추출
drawing_count_text = drawing_count.text
print(f"도면 개수: {drawing_count_text}")

# 브라우저 종료
driver.quit()

청구항 개수: 20
도면 개수: 21


In [90]:
# 요약서 추출
abstract = soup.find_all("div", class_="abstract")

print(abstract[0].text)

A device includes a first plurality of conductive strips have lengthwise directions in a first direction, a selector array overlapping the first plurality of conductive strips, an electrode array overlapping the selector array, a plurality of memory strips over the electrode array, and a second plurality of conductive strips overlapping the plurality of memory strips. The plurality of memory strips and the second plurality of conductive strips have lengthwise directions in a second direction perpendicular to the first direction.


In [91]:
# 청구항 추출
# 청구항 제1항 추출
claims1 = soup.select_one("#CLM-00001 > div")

print(claims1.text)
print(claim_count_number)

print(f"{int(claim_count_number):05d}" )

 1. A device comprising:
a first plurality of conductive strips having lengthwise directions parallel to a first direction; a first selector array overlapping the first plurality of conductive strips; a first electrode array overlapping the first selector array; a first plurality of memory strips over the first electrode array; and a second plurality of conductive strips overlapping the first plurality of memory strips, wherein the first plurality of memory strips and the second plurality of conductive strips have lengthwise directions parallel to a second direction perpendicular to the first direction. 
20
00020


In [92]:
claims = []

for i in range(1, int(claim_count_number)+1):
    formatted = f"{i:05d}"
    claims.append(soup.select_one(f"#CLM-{formatted} > div").text)

for claim in claims:
    print(claim)

 1. A device comprising:
a first plurality of conductive strips having lengthwise directions parallel to a first direction; a first selector array overlapping the first plurality of conductive strips; a first electrode array overlapping the first selector array; a first plurality of memory strips over the first electrode array; and a second plurality of conductive strips overlapping the first plurality of memory strips, wherein the first plurality of memory strips and the second plurality of conductive strips have lengthwise directions parallel to a second direction perpendicular to the first direction. 
 2. The device of claim 1 further comprising a plurality of conductive vias, each overlying and contacting one of the second plurality of conductive strips.
 3. The device of claim 1, wherein each of the first plurality of memory strips overlaps all selectors in an entire row of the first selector array.
 4. The device of claim 1 further comprising a plurality dielectric strips, wherei